In [1]:
import pandas as pd
import numpy as np



In [2]:
programs = []

for i in range(1, 9):
    program = pd.read_csv('../data/Program0%d.csv' % i)
    
    programs.append(program)
    
questions = pd.read_csv('../data/Question.csv')

In [3]:
import jieba 
#jieba.set_dictionary('../data/big5_dict.txt')

In [4]:
def jieba_line(lines):
    cut_lines = []
    for line in lines:
        cut_line = jieba.lcut(line)
        cut_lines.append(cut_line)
        
    return cut_lines

In [5]:
cut_programs = []

for program in programs:
    # number of lines in the program
    n = len(program)
    cut_program = []
    
    for i in range(n):
        lines = program.loc[i]['Content'].split('\n')
        cut_program.append(jieba_line(lines))
        
    cut_programs.append(cut_program)



Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/p6/qgv6p0s53svftjnzmh6nsr7h0000gn/T/jieba.cache
Loading model cost 1.264 seconds.
Prefix dict has been built succesfully.


In [6]:
cut_questions = []
n = len(questions)

for i in range(n):
    cut_line = []
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    for l in lines:
        cut_line.append(jieba.lcut(l))
    
    cut_question.append(cut_line)
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)
    


In [7]:
np.save('../data/cut_Programs.npy', cut_programs)
np.save('../data/cut_Questions.npy', cut_questions)

In [8]:
stopwordset = set()
cut_programs = np.load('../data/cut_Programs.npy')
cut_questions = np.load('../data/cut_Questions.npy')
#with open('../data/stopwords.txt', 'r', encoding='utf-8') as sw:
#    for line in sw:
#        stopwordset.add(line.strip('\n'))
        
stopwordset.add('\u3000')
stopwordset.add('\t')
stopwordset.add(' ')

In [9]:
program_out = open('program_quesiton.txt', 'w')
text_num = 0

for program in cut_programs:
    for episode in program:
        for line in episode:
            ex_line = ''
            for w in line:
                if w not in stopwordset:
                    ex_line += w + ' '
            program_out.write(ex_line+'\n')
            text_num += 1
            if (text_num % 100000 == 0):
                print ("Finished %d lines" % text_num)

print (text_num)

Finished 100000 lines
Finished 200000 lines
Finished 300000 lines
Finished 400000 lines
Finished 500000 lines
Finished 600000 lines
Finished 700000 lines
Finished 800000 lines
Finished 900000 lines
Finished 1000000 lines
Finished 1100000 lines
Finished 1200000 lines
Finished 1300000 lines
Finished 1400000 lines
Finished 1500000 lines
Finished 1600000 lines
Finished 1700000 lines
Finished 1800000 lines
Finished 1900000 lines
Finished 2000000 lines
Finished 2100000 lines
Finished 2200000 lines
Finished 2300000 lines
2396906


In [10]:
q_count = 0
for question in cut_questions:
    for i, line in enumerate(question):
        if (i == 0):
            ex_line = ''
            for qline in question[i]:
                for w in qline:
                    if (w not in stopwordset):
                        ex_line += w + ' '
            program_out.write(ex_line+'\n')
            q_count += 1
        else:
            ex_line = ''
            for w in line:
                if w not in stopwordset:
                    ex_line += w + ' '
            program_out.write(ex_line+'\n')
            q_count += 1
            
print (q_count)
program_out.close()

3500


In [11]:
from gensim.models import word2vec

sentences = word2vec.LineSentence('program_quesiton.txt')

model = word2vec.Word2Vec(sentences, size=250, window=5, min_count=1)

In [12]:
model.save('word2vec_model.bin')